# Student Number: 19200662

 #### Task 1: Select three review categories of your choice. Scrape all reviews for each category and store them as three separate datasets. For each review, you should store the review text and a class label (i.e. whether the review is “positive” or “negative”).

In [1]:
# import all the libraries required for the assignment
import bs4
import urllib.request
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import *
from sklearn.model_selection import cross_val_score, RepeatedKFold

In [2]:
#Define function to open the 3 different category url, read their content and store in separate dataframes

automotive_url = "http://mlg.ucd.ie/modules/yalp/automotive_list.html"
hairsalon_url ="http://mlg.ucd.ie/modules/yalp/hair_salons_list.html"
cafes_url= "http://mlg.ucd.ie/modules/yalp/cafes_list.html"

def get_url(url):
    response = urllib.request.urlopen(url)
    html = response.read().decode()
#     parser = bs4.BeautifulSoup(html,"html.parser")
    return html

# print(parser.prettify())
# print(soup)

In [3]:
# Get and parse data from Automotive category
soup= get_url(automotive_url)
parser_auto = bs4.BeautifulSoup(soup,"html.parser")

# Get and parse data from Cafes category
soup= get_url(cafes_url)
parser_cafes = bs4.BeautifulSoup(soup,"html.parser")

# Get and parse data from Hairsalon category
soup= get_url(hairsalon_url)
parser_hair = bs4.BeautifulSoup(soup,"html.parser")

In [4]:
# define a function to find 'a' tags to get the links of the businesses
def get_business_urls(parser):
    temp_business_urls =[]
    base_url = "http://mlg.ucd.ie/modules/yalp/"
    business_list = parser.find(id= 'all')
    # print(business_list)
    for match in business_list.find_all('a'):
        link = match.get('href')
        temp_business_urls.append(base_url+link)
    return temp_business_urls

business_urls_auto = get_business_urls(parser_auto)   # returns list of urls for automotive category
business_urls_cafes = get_business_urls(parser_cafes) # returns list of urls for cafes category
business_urls_hair = get_business_urls(parser_hair)   # returns list of urls for hairsalon category

print("Automotive category has %d businesses" %len(business_urls_auto),'\n',
      "Cafes category has %d businesses" %len(business_urls_cafes),'\n',
      "Hairsalon category has %d businesses" %len(business_urls_hair))

# print(len(business_urls))

# for bus in get_business_urls():
#     print(bus)

Automotive category has 132 businesses 
 Cafes category has 96 businesses 
 Hairsalon category has 143 businesses


In [5]:
# Define a function to open the html pages above and read content

review_data = []
def open_html(business_urls):
    
    headers = []
    headers.append("title")
    headers.append("review_text")
    headers.append("rating")
    review_data.append(headers)
    for b_url in business_urls:
        response = urllib.request.urlopen(b_url)
        html = response.read().decode()
        soup = bs4.BeautifulSoup(html,"html.parser")
#         print(soup)
        title = soup.find('h3',{'class': 'info'}).get_text()
#         print(title)
        reviews = soup.find(id ='all')
        
        for review in reviews.find_all(class_='review'):
            content =[]
            review_text = review.find ('p',{'class':'review-text'}).get_text()
            # print(review_text)
            rating_text = review.find('img')['alt']
            if rating_text == '4-star':
                rating = "positive"
            elif rating_text == '5-star':
                rating = "positive"
            elif rating_text == '1-star':
                rating = "negative"
            elif rating_text == '2-star':
                rating = "negative"
            elif rating_text == '3-star':
                rating = "negative"
            else:
                #for any other rating the value is assumed to be negative
                rating = "negative"
#             print(rating)
            content.append(title)
            content.append(review_text)
            content.append(rating)  
            review_data.append(content)
    return


In [6]:
#Call the function to read review data of Automotive category

open_html(business_urls_auto)
print(review_data)


[['title', 'review_text', 'rating'], ['Yalp — 7-Eleven', 'The man that was working tonight (8-12-17) was so rude. A real jerk. I need some help with getting change for my hundred and he was just horrible with customer service. I didnt get his name, but normally really nice people work here. Wonder what happened.', 'negative'], ['Yalp — 7-Eleven', 'Chris is a very rude person. Gave me an attitude about him not having change. Some people should not go to work if there in a grumpy mood. It shows', 'negative'], ['Yalp — 7-Eleven', "One of my favorite gas station to stop at. The store is always clean and the staff is friendly. This location is convenient and has a red box outside the store. The have hot and ready to serve pizza and wings as well. One thing I love about this gas station I can't even recall a time where someone has come up to me and asked for money which is definitely a plus. The pumps are always clean and the parking lot is always kept up. I can say I actually look forward t

In [7]:
# Store the review_data for automotive business into pandas dataframe

column_names = review_data.pop(0)
df_automotive = pd.DataFrame(review_data,columns=column_names)
df_automotive.head(10)


,title,review_text,rating
0,Yalp — 7-Eleven,The man that was working tonight (8-12-17) was...,negative
1,Yalp — 7-Eleven,Chris is a very rude person. Gave me an attitu...,negative
2,Yalp — 7-Eleven,One of my favorite gas station to stop at. The...,positive
3,Yalp — 7-Eleven,Oh thank Heaven for Seven Eleven! I don't know...,negative
4,Yalp — 7-Eleven,Five stars because of the guy who works weekda...,positive
5,Yalp — AA Auto Care,I had the best experience here!!!!!! All the e...,positive
6,Yalp — AA Auto Care,My objective with this shop was to find a good...,negative
7,Yalp — AA Auto Care,Absolute worst service ever. Went in for oil c...,negative
8,Yalp — AA Auto Care,Love love love their customer service. I come ...,positive
9,Yalp — AA Auto Care,I took my friend to this shop to get her oil c...,negative


In [8]:
review_data.clear()  # Clear the data stored in the review_data so that it can further store the next category dataset

In [9]:
#Call the function to read review data of Cafes category
open_html(business_urls_cafes)
print(review_data)

[['title', 'review_text', 'rating'], ['Yalp — ABC Bakery Coffee Shop', "Pros: Lots of items you would not expect from a Chinese bakery Friendly owners Great Value Cons: Service can be slow when it's busy Having read the previous reviews, apparently the owners of this place changed recently. I never experienced this place with the previous owners so I can only base my reviews on how they are now. ABC is your standard Chinese bakery with a lot of hot food you can order as well. I have always seen this place whenever I come to same plaza for dim sum. It looks very small on the outside but there's a large amount of tables and space when you come inside. ABC has all the items you would expect from a Chinese bakery and then some. For breakfast, they have a couple of combos that of great value ion my opinion. I got a combo which included a beef brisket noodle, egg sandwich and a very large cold milk tea for $6.50. The beef brisket noodle had a great amount of brisket and a good amount of vegg

In [10]:
# Store the review_data for cafes business into pandas dataframe

column_names = review_data.pop(0)
df_cafes = pd.DataFrame(review_data,columns=column_names)
df_cafes.head(10)

,title,review_text,rating
0,Yalp — ABC Bakery Coffee Shop,Pros: Lots of items you would not expect from ...,positive
1,Yalp — ABC Bakery Coffee Shop,Best egg-tarts in town! There's really not muc...,positive
2,Yalp — ABC Bakery Coffee Shop,I've been to ABC Bakery a few times since I re...,negative
3,Yalp — ABC Bakery Coffee Shop,"FYI, Closed Monday's New ownership for about 1...",negative
4,Yalp — ABC Bakery Coffee Shop,The inside may not look like much but they mak...,positive
5,Yalp — ABC Bakery Coffee Shop,I usually come for the egg tart and it is one ...,positive
6,Yalp — ABC Bakery Coffee Shop,"Everything, I meant everything were good. Were...",positive
7,Yalp — ABC Bakery Coffee Shop,"It's more like 3 and a 1/2 stars, but if value...",negative
8,Yalp — ABC Bakery Coffee Shop,This place got to have the best Hong Kong styl...,positive
9,Yalp — ABC Bakery Coffee Shop,Make a quick stop here while waiting for dim s...,positive


In [11]:
review_data.clear() # Clear the data stored in the review_data so that it can further store the next category dataset

In [12]:
#Call the function to read review data of Hairsalon category
open_html(business_urls_hair)
print(review_data)

[['title', 'review_text', 'rating'], ["Yalp — Ab's Barber Shop", "One of the best barbershops I've been to, with great prices. I honestly couldn't be happier with the love that comes out of this place, the environment is quite amazing and comfortable. I remember the day I found this place, they've been a success with my cuts since day 1. Thank you.", 'positive'], ["Yalp — Ab's Barber Shop", 'Took my son in for a haircut. Barber was great he did exactly what I wanted. Very clean cut will definitely be coming in more often.', 'positive'], ["Yalp — Ab's Barber Shop", 'Walked in, said hi. The only barber in there did not have a courtesy to say hi back. He was too busy watching the soccer game while cutting the hair. Stood in front of the reception desk for few minutes. Nothing. Left and will not be back.', 'negative'], ["Yalp — Ab's Barber Shop", "I came here 10 minutes before 9am to get a haircut. They are open at 9am on Saturday's according to their hours of operation. It is now 9:15 and

In [13]:
# Store the review_data for hairsalon business into pandas dataframe

column_names = review_data.pop(0)
df_hairsalon = pd.DataFrame(review_data,columns=column_names)
df_hairsalon.head(10)

,title,review_text,rating
0,Yalp — Ab's Barber Shop,"One of the best barbershops I've been to, with...",positive
1,Yalp — Ab's Barber Shop,Took my son in for a haircut. Barber was great...,positive
2,Yalp — Ab's Barber Shop,"Walked in, said hi. The only barber in there d...",negative
3,Yalp — Ab's Barber Shop,I came here 10 minutes before 9am to get a hai...,negative
4,Yalp — Ab's Barber Shop,"Great haircut. No fuss no muss, I asked for la...",positive
5,Yalp — Adore Salon and Boutique,Love this place great atmosphere and energy. V...,positive
6,Yalp — Adore Salon and Boutique,Loved this salon. It is very clean an smells g...,positive
7,Yalp — Adore Salon and Boutique,I will never recommend this business to anyone...,negative
8,Yalp — Adore Salon and Boutique,I bought a Groupon from Sandra Lynn Rosendin f...,positive
9,Yalp — Adore Salon and Boutique,Ester is fabulous with hair. She colors and no...,positive


In [14]:
review_data.clear() # Clear the data stored in the review_data so that it can further store the next category dataset

#### Task 1 Conclusion: 

*Three dataframes created are:*
    *Category A = df_automotive*, 
    *Category B =df_cafes*,
    *Category C= df_hairsalon*
   

#### Task 2: For each of the three category datasets:

#### ***a. From the reviews in this category, apply appropriate preprocessing steps to create a numeric representation of the data, suitable for classification.***

#### Data Pre-processing

In [15]:
# Category A: Automotive dataset

# df_automotive.describe()
df_automotive.isnull().sum()  # check if there is null value in the dataframe

title          0
review_text    0
rating         0
dtype: int64

In [16]:
# document and target creation for automotive dataset
documents_auto = df_automotive["review_text"]
target_auto = df_automotive["rating"]
# target_auto.shape

In [17]:
# Category B: Cafes dataset

# df_hairsalon.describe()
df_cafes.isnull().sum()  # check if there is null value in the dataframe

title          0
review_text    0
rating         0
dtype: int64

In [18]:
# document and target creation for cafes dataset
documents_cafes = df_cafes["review_text"]
target_cafes = df_cafes["rating"]

In [19]:
# Category C: Hairsalon dataset

# df_hairsalon.describe()
df_hairsalon.isnull().sum()  # check if there is null value in the dataframe

title          0
review_text    0
rating         0
dtype: int64

In [20]:
# document and target creation for hairsalon dataset

documents_hairsalon = df_hairsalon["review_text"]
target_hairsalon = df_hairsalon["rating"]

In [21]:
# check class bifurcation for each category of the target dataset

print("For Automotive: " , target_auto.value_counts())

print("For Hairsalon: ",target_hairsalon.value_counts())

print("For Cafes:", target_cafes.value_counts())

For Automotive:  positive    1212
negative     788
Name: rating, dtype: int64
For Hairsalon:  positive    1558
negative     442
Name: rating, dtype: int64
For Cafes: positive    1462
negative     538
Name: rating, dtype: int64


In [22]:
# Produce vector representation of each of the documents set 

from sklearn.feature_extraction.text import TfidfVectorizer
# perform preprocessing to create a document-term matrix
vectorizer = TfidfVectorizer(stop_words="english",min_df = 10)

auto = vectorizer.fit_transform(documents_auto) # Automotive dataset
cafes = vectorizer.fit_transform(documents_cafes) # Cafes dataset
hairsalon = vectorizer.fit_transform(documents_hairsalon) #Hairsalon dataset


print("Shape of vectorized category A(Automotive) data is :", auto.shape)
print("Shape of vectorized category B(Cafes) data is :", cafes.shape)
print("Shape of vectorized category C(Hairsalon) data is :", hairsalon.shape)



Shape of vectorized category A(Automotive) data is : (2000, 1500)
Shape of vectorized category B(Cafes) data is : (2000, 1496)
Shape of vectorized category C(Hairsalon) data is : (2000, 1285)


In [23]:
# check the distinct terms in each of the vectors

terms_auto = vectorizer.get_feature_names()
terms_hairsalon = vectorizer.get_feature_names()
terms_cafes= vectorizer.get_feature_names()
print("Vocabulary has %d distinct auto terms" % len(terms_auto),'\n',
     "Vocabulary has %d distinct hairsalon terms" % len(terms_hairsalon),'\n',
     "Vocabulary has %d distinct cafes terms" % len(terms_cafes))


Vocabulary has 1285 distinct auto terms 
 Vocabulary has 1285 distinct hairsalon terms 
 Vocabulary has 1285 distinct cafes terms


In [24]:
# Display a set of sample terms
print("Automotive:", terms_auto[20:30])
print("Cafes:", terms_cafes[20:30])
print("Hairsalon:", terms_hairsalon[20:30])

Automotive: ['absolute', 'absolutely', 'accept', 'accommodate', 'accommodating', 'achieve', 'actual', 'actually', 'add', 'added']
Cafes: ['absolute', 'absolutely', 'accept', 'accommodate', 'accommodating', 'achieve', 'actual', 'actually', 'add', 'added']
Hairsalon: ['absolute', 'absolutely', 'accept', 'accommodate', 'accommodating', 'achieve', 'actual', 'actually', 'add', 'added']


#### *b. Build a classification model to distinguish between “positive” and “negative” reviews using one of the following classifiers: Naive Bayes, Logistic Regression, Random Forests***

#### *c. Test the predictions of the classification model using an appropriate evaluation strategy. Report and discuss the evaluation results in your notebook.*

#### Classification and Evaluation
*Build a NAIVE BAYES classification model, and evaluate its performance using a single training/test split by taking 70:30 training and test set*

In [25]:
# Split the data for automotive category:

data_train, data_test, target_train, target_test = train_test_split(auto, target_auto, test_size=0.3,random_state = 0)

print("Training set has %d examples" % data_train.shape[0] )
print("Test set has %d examples" % data_test.shape[0] )

# Prepare the NAIVE BAYES classification model for automotive category

from sklearn.naive_bayes import MultinomialNB
model=  MultinomialNB()
model.fit(data_train,target_train)

#Make predictions for the reviews in the test data
predicted = model.predict(data_test)
predicted[0:5]

Training set has 1400 examples
Test set has 600 examples


array(['negative', 'positive', 'positive', 'positive', 'positive'],
      dtype='<U8')

In [26]:
# Get the accuracy of the prediction

print("Accuracy of Automotive dataset model = %.2f" % accuracy_score(target_test, predicted) )


Accuracy of Automotive dataset model = 0.91


In [27]:
# Get the confusion matrix of the prediction
cm = confusion_matrix(target_test, predicted,labels=['negative','positive'])
print(cm)

[[203  40]
 [ 14 343]]


In [28]:
# Get an overall classification report of the Automotive dataset
auto_classification_report=classification_report(target_test, predicted, target_names=["negative","positive"])
print(auto_classification_report)

              precision    recall  f1-score   support

    negative       0.94      0.84      0.88       243
    positive       0.90      0.96      0.93       357

    accuracy                           0.91       600
   macro avg       0.92      0.90      0.90       600
weighted avg       0.91      0.91      0.91       600



In [29]:
# Split the data for Cafes category:

data_train, data_test, target_train, target_test = train_test_split(cafes, target_cafes, test_size=0.3,random_state = 0)

print("Training set has %d examples" % data_train.shape[0] )
print("Test set has %d examples" % data_test.shape[0] )

# Prepare the NAIVE BAYES classification model for cafes category

from sklearn.naive_bayes import MultinomialNB
model=  MultinomialNB()
model.fit(data_train,target_train)

#Make predictions for the reviews in the test data
predicted = model.predict(data_test)
predicted[0:5]

Training set has 1400 examples
Test set has 600 examples


array(['negative', 'positive', 'positive', 'positive', 'positive'],
      dtype='<U8')

In [30]:
# Get the accuracy of the prediction 

print("Accuracy of Cafes dataset model = %.2f" % accuracy_score(target_test, predicted) )


Accuracy of Cafes dataset model = 0.84


In [31]:
#Get the confusion matrix of the prediction

cm = confusion_matrix(target_test, predicted,labels=['negative','positive'])
print(cm)

[[ 57  81]
 [ 13 449]]


In [32]:
# Get an overall classification report of the Cafes dataset
cafe_classification_report=classification_report(target_test, predicted, target_names=["negative","positive"])
print(cafe_classification_report)

              precision    recall  f1-score   support

    negative       0.81      0.41      0.55       138
    positive       0.85      0.97      0.91       462

    accuracy                           0.84       600
   macro avg       0.83      0.69      0.73       600
weighted avg       0.84      0.84      0.82       600



In [33]:
# Split the data for hairsalon category:

data_train, data_test, target_train, target_test = train_test_split(hairsalon, target_hairsalon, test_size=0.3,random_state = 0)
print("Training set has %d examples" % data_train.shape[0] )
print("Test set has %d examples" % data_test.shape[0] )


# Prepare the NAIVE BAYES classification model for hairsalon category

from sklearn.naive_bayes import MultinomialNB
model=  MultinomialNB()
model.fit(data_train,target_train)

#Make predictions for the reviews in the test data
predicted = model.predict(data_test)
predicted[0:5]

Training set has 1400 examples
Test set has 600 examples


array(['positive', 'positive', 'positive', 'positive', 'positive'],
      dtype='<U8')

In [34]:
#Get the accuracy of the predictions

print("Accuracy of the hairsalon dataset model = %.2f" % accuracy_score(target_test, predicted) )

Accuracy of the hairsalon dataset model = 0.88


In [35]:
# Get the confusion matrix of the predictions
cm = confusion_matrix(target_test, predicted,labels=['negative','positive'])
print(cm)

[[ 63  68]
 [  3 466]]


In [36]:
# Get an overall classification report of the hairsalon dataset
hairsalon_classification_report=classification_report(target_test, predicted, target_names=["negative","positive"])
print(hairsalon_classification_report)

              precision    recall  f1-score   support

    negative       0.95      0.48      0.64       131
    positive       0.87      0.99      0.93       469

    accuracy                           0.88       600
   macro avg       0.91      0.74      0.78       600
weighted avg       0.89      0.88      0.87       600



#### Conclusion Task 2:
The Classification and Evaluation was done using Naive Bayes model for three business categories- Automotive, hairsalon and Cafes and it is observed that cafes model has the least accuracy(84%) and the other two business category have nearly 90% prediction accuracy.
Amongst the other two categories Automotive data set has the highest prediction accuracy of 91% which can act as suitable training data set to build a model for machine learning.

#### Task 3: Evaluate the performance of each of your three classification models when applied to data from the other two selected categories. That is, for the selected categories (A,B,C), run the experiments:


#### *a. Train a classification model on the data from “Category A”. Evaluate its performance on data from “Category B” and data from “Category C”.* 

In [37]:
#Classification and pre-processing of category A data is performed below:
# Training data: category A: Automotive    

from sklearn.feature_extraction.text import TfidfVectorizer
# perform preprocessing to the training data to create a document-term matrix
vectorizer = TfidfVectorizer(stop_words="english",min_df = 10)
X = vectorizer.fit_transform(documents_auto)
print("Shape of vectorized category A data is :", X.shape)
terms = vectorizer.get_feature_names()
print("Vocabulary has %d distinct terms" % len(terms))

# Use naive bayes classifiers and build model on the training the data

from sklearn.naive_bayes import MultinomialNB
model=  MultinomialNB()
#Make predictions for the reviews in the test set.
model.fit(X, target_auto)
# print(model)

#Performance evaluation of cataegory A data basis category B data is performed below:
# Test data= category B: Cafes

#vectorize the test data into document -term matrix using transform() function
test_B = vectorizer.transform(documents_cafes)
print("Shape of vectorized category B data is",test_B.shape)

# predict and evaluate the test data using trained model

predicted = model.predict(test_B)
from sklearn.metrics import accuracy_score
acc = accuracy_score(target_cafes, predicted)
print("Classification accuracy of category B(Cafes) data = %.2f" % acc)


#Performance evaluation of cataegory A data basis category C data is performed below:
# Test data= category C: Hairsalon

#vectorize the test data into document -term matrix using transform() function
test_C = vectorizer.transform(documents_hairsalon)
print("shape of vectorized category C data is",test_C.shape)

# predict and evaluate the test data using trained model

predicted = model.predict(test_C)
from sklearn.metrics import accuracy_score
acc = accuracy_score(target_hairsalon, predicted)
print("Classification accuracy of category C(Hairsalon) data = %.2f" % acc)


Shape of vectorized category A data is : (2000, 1500)
Vocabulary has 1500 distinct terms
Shape of vectorized category B data is (2000, 1500)
Classification accuracy of category B(Cafes) data = 0.83
shape of vectorized category C data is (2000, 1500)
Classification accuracy of category C(Hairsalon) data = 0.92


#### *b. Train a classification model on the data from “Category B”. Evaluate its performance on data from “Category A” and data from “Category C”.*

In [38]:
# Training data: category B: Cafes   

from sklearn.feature_extraction.text import TfidfVectorizer
# perform preprocessing to the training data to create a document-term matrix
vectorizer = TfidfVectorizer(stop_words="english",min_df = 10)
X = vectorizer.fit_transform(documents_cafes)
print("Shape of vectorized category B data is :", X.shape)
terms = vectorizer.get_feature_names()
print("Vocabulary of category B has %d distinct terms" % len(terms))

# Use naive bayes classifiers and build model on the training the data

from sklearn.naive_bayes import MultinomialNB
model=  MultinomialNB()
#Make predictions for the reviews in the test set.
model.fit(X, target_cafes)
print("Model B:", model)


#Performance evaluation of category B data basis category A data is performed below: 
# Test data= category A: Automotive

#vectorize the test data into document -term matrix using transform() function
test_A = vectorizer.transform(documents_auto)
print("Shape of vectorized category A data is",test_A.shape)

# predict and evaluate the test data using trained model

predicted = model.predict(test_A)
from sklearn.metrics import accuracy_score
acc = accuracy_score(target_auto, predicted)
print("Classification accuracy of category A(Auto) data = %.2f" % acc)

#Performance evaluation of category B data basis category C data is performed below: 
# Test data= category C: Hairsalon

#vectorize the test data into document -term matrix using transform() function
test_C = vectorizer.transform(documents_hairsalon)
print("shape of vectorized category C data is",test_C.shape)

# predict and evaluate the test data using trained model

predicted = model.predict(test_C)
from sklearn.metrics import accuracy_score
acc = accuracy_score(target_hairsalon, predicted)
print("Classification accuracy of category C(Hairsalon) data = %.2f" % acc)

Shape of vectorized category B data is : (2000, 1496)
Vocabulary of category B has 1496 distinct terms
Model B: MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)
Shape of vectorized category A data is (2000, 1496)
Classification accuracy of category A(Auto) data = 0.80
shape of vectorized category C data is (2000, 1496)
Classification accuracy of category C(Hairsalon) data = 0.90


#### *c. Train a classification model on the data from “Category C”. Evaluate its performance on data from “Category A” and data from “Category B”.* 

In [39]:
# Training data: category C: Hairsalon 

from sklearn.feature_extraction.text import TfidfVectorizer
# perform preprocessing to the training data to create a document-term matrix
vectorizer = TfidfVectorizer(stop_words="english",min_df = 10)
X = vectorizer.fit_transform(documents_hairsalon)
print("Shape of vectorized category C data is :", X.shape)
terms = vectorizer.get_feature_names()
print("Vocabulary of category C has %d distinct terms" % len(terms))

# Use naive bayes classifiers and build model on the training the data

from sklearn.naive_bayes import MultinomialNB
model=  MultinomialNB()
#Make predictions for the reviews in the test set.
model.fit(X, target_hairsalon)
print("Model C:", model)


#Performance evaluation of category C data basis category A data is performed below: 
# Test data= category A: Automotive

#vectorize the test data into document -term matrix using transform() function
test_A = vectorizer.transform(documents_auto)
print("Shape of vectorized category A data is",test_A.shape)

# predict and evaluate the test data using trained model

predicted = model.predict(test_A)
from sklearn.metrics import accuracy_score
acc = accuracy_score(target_auto, predicted)
print("Classification accuracy of category A(Auto) data = %.2f" % acc)

#Performance evaluation of cataegory C data basis category B data is performed below: 
# Test data= category C: Hairsalon

#vectorize the test data into document -term matrix using transform() function
test_B = vectorizer.transform(documents_cafes)
print("shape of vectorized category B data is",test_B.shape)

# predict and evaluate the test data using trained model

predicted = model.predict(test_B)
from sklearn.metrics import accuracy_score
acc = accuracy_score(target_cafes, predicted)
print("Classification accuracy of category B(Cafes) data = %.2f" % acc)

Shape of vectorized category C data is : (2000, 1285)
Vocabulary of category C has 1285 distinct terms
Model C: MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)
Shape of vectorized category A data is (2000, 1285)
Classification accuracy of category A(Auto) data = 0.84
shape of vectorized category B data is (2000, 1285)
Classification accuracy of category B(Cafes) data = 0.81


#### Conclusion Task 3:

*Summary of the accuracy results of each of the categories when acted as training dataset and tested on other datasets is shown below:*

Category A|Category B|Category C
----------|----------|----------
Train_data| 83%      |   92%
80%       |Train_data| 90%
84%       |  81%     | Train_data

We can see that Category A has ~87% prediction accuracy, Category B has ~85% accuracy and Category C has the least ~82% accuracy.
The results suggests that Catagory dataset(Automotive) can be a suitable fit for Machine learning algor ithm(havingthe highest accuracy amongst the three category). Moreover, the same prediction model can be further build using other classification models and can be tested for better accuracy results. However, KNN classification model was also checked but it showed lesser accuracy hence Naive Bayes model is found to be suitable for the given assignment.